In [98]:
import os
import random
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

ROOT_DIR = "./data"
RANDOM_STATE = 110

In [99]:
# read csv
df = pd.read_excel(os.path.join(ROOT_DIR, "Dam dispensing.xlsx"), header=[0,1])
# df = pd.read_csv(os.path.join(ROOT_DIR, "Dam dispensing.csv"), low_memory=False)
df.head()

KeyboardInterrupt: 

In [ ]:
df.columns

Index(['Wip Line', 'Process Desc.', 'Equipment', 'Model.Suffix', 'Workorder',
       'LOT ID', 'Set ID', 'Box ID', 'Collect Date', 'Insp. Seq No.',
       ...
       'Collect Result.67', 'Unit Time.67', 'Judge Value.67',
       'Collect Result.68', 'Unit Time.68', 'Judge Value.68',
       'Collect Result.69', 'Unit Time.69', 'Judge Value.69', 'Unnamed: 221'],
      dtype='object', length=222)

In [ ]:
nan_columns = df.columns[df.isnull().any()]
# 전부 NaN이 아니더라도 최소 2000개의 NaN이라서 전부 삭제

In [ ]:
df.drop(nan_columns, axis=1, inplace=True)
df.drop(["Process Desc.", "LOT ID"], axis=1, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62479 entries, 0 to 62478
Data columns (total 77 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Wip Line           62479 non-null  object 
 1   Equipment          62479 non-null  object 
 2   Model.Suffix       62479 non-null  object 
 3   Workorder          62479 non-null  object 
 4   Set ID             62479 non-null  object 
 5   Collect Date       62479 non-null  object 
 6   Insp. Seq No.      62479 non-null  int64  
 7   Insp Judge Code    62479 non-null  object 
 8   Collect Result     62479 non-null  float64
 9   Collect Result.1   62479 non-null  float64
 10  Collect Result.2   62479 non-null  int64  
 11  Collect Result.3   62479 non-null  int64  
 12  Collect Result.4   62479 non-null  int64  
 13  Collect Result.5   62479 non-null  float64
 14  Collect Result.6   62479 non-null  int64  
 15  Collect Result.7   62479 non-null  int64  
 16  Collect Result.8   624

In [ ]:
df.describe()

,Insp. Seq No.,Collect Result,Collect Result.1,Collect Result.2,Collect Result.3,Collect Result.4,Collect Result.5,Collect Result.6,Collect Result.7,Collect Result.8,...,Collect Result.60,Collect Result.61,Collect Result.62,Collect Result.63,Collect Result.64,Collect Result.65,Collect Result.66,Collect Result.67,Collect Result.68,Collect Result.69
count,62479.000000,62479.000000,62479.000000,62479.000000,62479.000000,62479.0,62479.0,62479.0,62479.000000,62479.0,...,62479.000000,62479.000000,62479.000000,62479.000000,62479.000000,62479.000000,62479.000000,62479.000000,62479.000000,62479.000000
mean,1.055491,526.257695,6.328646,-21.216889,73.616559,1150.0,33.5,0.0,747.526689,33.5,...,6522.093824,6522.093824,6516.011780,6519.620993,6516.011780,6518.476608,3466.795243,-0.001662,-0.008205,2.894915
std,0.247805,367.155839,4.869219,87.464084,9.945053,0.0,0.0,0.0,362.329333,0.0,...,1752.201752,1752.201752,1747.053612,1752.583970,1747.053612,1752.759645,3045.446957,0.009546,0.033493,3.452136
min,1.000000,240.000000,2.500000,-90.000000,32.000000,1150.0,33.5,0.0,280.000000,33.5,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,0.000000,-0.054000,-0.219000,-0.118000
25%,1.000000,240.000000,2.500000,-90.000000,70.000000,1150.0,33.5,0.0,280.000000,33.5,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,240.000000,2.500000,-90.000000,70.000000,1150.0,33.5,0.0,1030.000000,33.5,...,6500.000000,6500.000000,6500.000000,6500.000000,6500.000000,6500.000000,4000.000000,0.000000,0.000000,0.000000
75%,1.000000,1000.000000,12.500000,90.000000,70.000000,1150.0,33.5,0.0,1030.000000,33.5,...,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,6500.000000,0.000000,0.000000,7.000000
max,4.000000,1000.000000,13.500000,90.000000,105.000000,1150.0,33.5,0.0,1030.000000,33.5,...,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,0.037000,0.007000,7.000000


In [ ]:
df["Equipment"].unique()

array(['Dam dispenser #1', 'Dam dispenser #2'], dtype=object)

In [ ]:
df["Equipment"] = df["Equipment"].str.split("#", expand=True)[1]

In [ ]:
df.head()

,Wip Line,Equipment,Model.Suffix,Workorder,Set ID,Collect Date,Insp. Seq No.,Insp Judge Code,Collect Result,Collect Result.1,...,Collect Result.60,Collect Result.61,Collect Result.62,Collect Result.63,Collect Result.64,Collect Result.65,Collect Result.66,Collect Result.67,Collect Result.68,Collect Result.69
0,IVI-OB6,1,AJX75334501,3F1X5847-2,OP753345013050000002,2023-05-04 08:57:23,1,OK,240.0,2.5,...,9000,9000,7000,9000,7000,9000,0,0.0,0.0,7.0
1,IVI-OB6,1,AJX75334501,3F1X5847-2,OP753345013050000003,2023-05-04 09:11:35,1,OK,240.0,2.5,...,9000,9000,7000,9000,7000,9000,0,0.0,0.0,7.0
2,IVI-OB6,1,AJX75334501,3F1X5847-2,OP753345013050000004,2023-05-04 09:13:19,1,OK,240.0,2.5,...,9000,9000,7000,9000,7000,9000,0,0.0,0.0,7.0
3,IVI-OB6,1,AJX75334501,3F1X5847-2,OP753345013050000005,2023-05-04 09:15:24,1,OK,240.0,2.5,...,9000,9000,7000,9000,7000,9000,0,0.0,0.0,7.0
4,IVI-OB6,1,AJX75334501,3F1X5847-2,OP753345013050000006,2023-05-04 09:17:27,1,OK,240.0,2.5,...,9000,9000,7000,9000,7000,9000,0,0.0,0.0,7.0


In [ ]:
df["Model.Suffix"].unique()

array(['AJX75334501', 'AJX75334503', 'AJX75334502', 'EAT65200901',
       'AJX75334505', 'AJX75334506', 'AJX75334507', 'AJX75334508'],
      dtype=object)

In [ ]:
df["Model.Suffix - 1"] = df["Model.Suffix"].str[:4]
# e.g. AJX7

In [ ]:
df["Workorder - 1"] = df["Workorder"].str.split('-', expand=True)[0].str[:4]
# e.g. 3FIX

In [ ]:
df["Collect Date"] = pd.to_datetime(df["Collect Date"], format="%Y-%m-%d %H:%M:%S")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62479 entries, 0 to 62478
Columns: 222 entries, ('Collect Item:', 'Wip Line') to ('WorkMode', 'Judge Value.1')
dtypes: datetime64[ns](1), float64(173), int64(38), object(10)
memory usage: 105.8+ MB


In [96]:
df["Collect Result.66"]

KeyError: 'Collect Result.66'